In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
from scipy.stats import norm, t

In [14]:
import pingouin
import seaborn as sns

In [3]:
stackOVerF = pd.read_feather("C:/Users/User/Downloads/Data Ana/HypothesisD/stack_overflow.feather")

In [4]:
late_shipments = pd.read_feather("C:/Users/User/Downloads/Data Ana/HypothesisD/late_shipments.feather")

In [5]:
dem_votes_potus_12_16 = pd.read_feather("C:/Users/User/Downloads/Data Ana/HypothesisD/dem_votes_potus_12_16.feather")

#### Republic Can Votes

In [6]:
dem_votes_potus_12_16['diff'] = dem_votes_potus_12_16['dem_percent_12'] - dem_votes_potus_12_16['dem_percent_16']

In [ ]:
# Find the mean of the diff column
xbar_diff = dem_votes_potus_12_16['diff'].mean()
s_diff = dem_votes_potus_12_16['diff'].std()
print(xbar_diff, s_diff)

In [ ]:

dem_votes_potus_12_16['diff'].hist(bins=20)

In [11]:
# Conduct a t-test on diff
test_results = pingouin.ttest(x=dem_votes_potus_12_16['diff'], y=0, alternative="two-sided")

# Conduct a paired t-test on dem_percent_12 and dem_percent_16
paired_test_results = pingouin.ttest(x=sample_dem_data['dem_percent_12'], 
                              y=sample_dem_data['dem_percent_16'], paired=True,
                              alternative="two-sided")
                              
# Print the test results
print(test_results)

                T  dof alternative          p-val         CI95%   cohen-d  \
T-test  30.298384  499   two-sided  3.600634e-115  [6.39, 7.27]  1.354985   

              BF10  power  
T-test  2.246e+111    1.0  


## Anova test

##### late shipment

In [ ]:
# Calculate the mean pack_price for each shipment_mode
xbar_pack_by_mode = late_shipments.groupby('shipment_mode')['pack_price'].mean()

# Calculate the standard deviation of the pack_price for each shipment_mode
s_pack_by_mode = late_shipments.groupby("shipment_mode")['pack_price'].std()

sns.boxplot(x="pack_price", y="shipment_mode", data=late_shipments)

plt.show()
# Run an ANOVA for pack_price across shipment_mode
anova_results = pingouin.anova(data=late_shipments, dv='pack_price', between='shipment_mode')

# Modify the pairwise t-tests to use Bonferroni p-value adjustment
pairwise_results = pingouin.pairwise_tests(data=late_shipments, 
                                           dv="pack_price",
                                           between="shipment_mode",
                                           padjust="bonf")

# Print pairwise_results
print(pairwise_results)
# Print the grouped means

print(xbar_pack_by_mode)

#### StackOverflow Testings

In [5]:
boot_strapDist = []
for i in range(10000):
    boot_strapDist.append(
        np.mean(stackOVerF.sample(frac=1, replace=True)['converted_comp'])
    )

#### Late shipment Tests

In [6]:
late_prop_samp = (late_shipments['late'] == 'Yes').mean()

In [15]:
late_shipments['ProsHelper'] = late_shipments['late'] == 'Yes'

In [8]:

late_bootStrapDis = []
for i in range(5000):
    late_bootStrapDis.append(
        np.mean(late_shipments.sample(frac=1, replace=True)['ProsHelper'])
    )

#### Calculating z - score

In [13]:
# Hypothesize that the proportion is 6%
late_prop_hyp = 0.06

# Calculate the standard error
std_error = np.std(late_bootStrapDis, ddof=1)

# Find z-score of late_prop_samp
z_score = (late_prop_samp - late_prop_hyp)/std_error

# Print z_score
p_val = 1 - norm.cdf(z_score, loc=0, scale=1)
print(z_score, p_val)

0.13246842393559696 0.44730689847132943


### Using one proportion

In [ ]:
# Hypothesize that the proportion of late shipments is 6%
p_0 = 0.06

# Calculate the sample proportion of late shipments
p_hat = (late_shipments['late'] == "Yes").mean()

# Calculate the sample size
n = len(late_shipments)

# Calculate the numerator and denominator of the test statistic
numerator = p_hat - p_0
denominator = np.sqrt(p_0 * (1 - p_0) / n)

# Calculate the test statistic
z_score = numerator / denominator

# Calculate the p-value from the z-score
p_value = 1 - norm.cdf(z_score)
# Print the p-value
print(p_value)

In [45]:
p_por = late_shipments[late_shipments['late'] == 'Yes'].groupby(['freight_cost_groups', 'late']).std()

In [46]:
p_por

,,id,late_delivery,unit_of_measure_per_pack,line_item_quantity,line_item_value,pack_price,unit_price,weight_kilograms,freight_cost_usd,line_item_insurance_usd,ProsHelper
freight_cost_groups,late,,,,,,,,,,,
expensive,Yes,22160.740187,0.0,40.557239,31191.457022,206136.167946,38.578512,0.388325,2401.284638,15913.016376,334.579319,0.0
reasonable,Yes,22890.163111,0.0,68.529191,29695.766742,187243.958299,16.621680,0.174317,3011.154872,1425.285071,404.738017,0.0


In [51]:
ns = late_shipments.groupby('freight_cost_groups')['late'].count()

#### test of two propotion test

In [ ]:
# Calculate the pooled estimate of the population proportion
p_hat = (p_hats["reasonable"] * ns["reasonable"] + p_hats["expensive"] * ns["expensive"]) / (ns["reasonable"] + ns["expensive"])

# Calculate p_hat one minus p_hat
p_hat_times_not_p_hat = p_hat * (1 - p_hat)

# Divide this by each of the sample sizes and then sum
p_hat_times_not_p_hat_over_ns = p_hat_times_not_p_hat / ns["expensive"] + p_hat_times_not_p_hat / ns["reasonable"]

# Calculate the standard error
std_error = np.sqrt(p_hat_times_not_p_hat_over_ns)

# Calculate the z-score
z_score = (p_hats["expensive"] - p_hats["reasonable"]) / std_error

# Calculate the p-value from the z-score
p_value = 1- norm.cdf(z_score)

# Print p_value
print(p_value)

In [ ]:
# Count the late column values for each freight_cost_group
late_by_freight_cost_group = late_shipments.groupby("freight_cost_group")['late'].value_counts()

# Create an array of the "Yes" counts for each freight_cost_group
success_counts = np.array([
late_by_freight_cost_group[:][1],
late_by_freight_cost_group[:][3]])

# Create an array of the total number of rows in each freight_cost_group
n = np.array([late_shipments.groupby('freight_cost_group')['late'].count()[0], late_shipments.groupby('freight_cost_group')['late'].count()[1]])

# Run a z-test on the two proportions
stat, p_value = proportions_ztest(count=success_counts,nobs=n, alternative="larger")


# Print the results
print(stat, p_value)

#### Chi Square Independent Test

In [ ]:
# Proportion of freight_cost_group grouped by vendor_inco_term
props = late_shipments.groupby('vendor_inco_term')['freight_cost_group'].value_counts(normalize=True)

# Convert props to wide format
wide_props = props.unstack()

# Proportional stacked bar plot of freight_cost_group vs. vendor_inco_term
wide_props.plot(kind="bar", stacked=True)
plt.show()

# Determine if freight_cost_group and vendor_inco_term are independent
expected, observed, stats = pingouin.chi2_independence(data=late_shipments, x="freight_cost_group", y="vendor_inco_term")

# Print results
print(stats[stats['test'] == 'pearson']) 

In [ ]:
# Find the number of rows in late_shipments
n_total = len(late_shipments)

# Create n column that is prop column * n_total
hypothesized["n"] = hypothesized["prop"] * n_total

# Plot a red bar graph of n vs. vendor_inco_term for incoterm_counts
plt.bar(incoterm_counts['vendor_inco_term'], incoterm_counts['n'], color="red", label="Observed")

# Add a blue bar plot for the hypothesized counts
plt.bar(hypothesized['vendor_inco_term'], hypothesized["n"],color="b", label="Hypothesized", alpha=0.5)
plt.legend()
plt.show()

# Perform a goodness of fit test on the incoterm counts n
gof_test = chisquare(f_obs=incoterm_counts["n"], f_exp=hypothesized["n"])


# Print gof_test results
print(gof_test)

In [52]:
ns

freight_cost_groups
expensive     531
reasonable    455
Name: late, dtype: int64

#### Calulating the T-stats for the two groups Yes/No
T test

 ----- When testing for differences between means, the test statistic is called 't' rather than 'z', and can be calculated using six numbers from the samples-----
 

In [50]:
xbar_no = (late_shipments[late_shipments['late'] == "No"]['weight_kilograms']).mean()
xbar_yes = (late_shipments[late_shipments['late'] == "Yes"]['weight_kilograms']).mean()
s_no = np.std(late_shipments[late_shipments['late'] == "No"]['weight_kilograms'])
s_yes = np.std(late_shipments[late_shipments['late'] == "Yes"]['weight_kilograms'])
n_no = len(late_shipments[late_shipments['late'] == "No"])
n_yes = len(late_shipments[late_shipments['late'] == "Yes"])

numerator = xbar_no - xbar_yes
denominator =  np.sqrt(s_no**2/n_no + s_yes**2/n_yes)

# Calculate the test statistic
t_stat = numerator/denominator

# Print the test statistic
print(t_stat)


-2.411826236645244


In [48]:
numerator = xbar_no - xbar_yes

In [49]:
denominator =  np.sqrt(s_no**2/n_no + s_yes**2/n_yes)

353.17867206833523

In [52]:
degrees_of_freedom = n_yes + n_no -2 

# Calculate the p-value from the test stat
p_value = t.cdf(t_stat, df=degrees_of_freedom)

# Print the p_value
print(p_value)

0.008026243633867782


#### 

In [ ]:
# Select the weight_kilograms and late columns
weight_vs_late = late_shipments[['weight_kilograms', 'late']]

# Convert weight_vs_late into wide format
weight_vs_late_wide = weight_vs_late.pivot(columns='late', 
                                           values='weight_kilograms')


# Run a two-sided Wilcoxon-Mann-Whitney test on weight_kilograms vs. late
wmw_test = pingouin.mwu(x=weight_vs_late_wide['No'], y=weight_vs_late_wide['Yes'], alternative='two-sided')



# Print the test results
print(wmw_test)

# Run a Kruskal-Wallis test on weight_kilograms vs. shipment_mode
kw_test = pingouin.kruskal(data=late_shipments, dv='weight_kilograms', between='shipment_mode')



# Print the results
print(kw_test)

#### @ Paired T test